<a href="https://colab.research.google.com/github/drmanuelhuerfano/SalidasColombianos/blob/main/Taller_Extracci%C3%B3n_repositorio_analisisMH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---


<h2>Taller Extracción y repositorio de análisis</h2>


---


**Preparación de datos**

**Especialización en Inteligencia Artificial**

**Pontificia Universidad Javeriana**

### **GRUPO 1**

### Integrantes:
Leidy Milena Carrillo Hernández - leidy.carrilloh@javeriana.edu.co <br>
Juan Gabriel Ibarra España - ju_ibarra@javeriana.edu.co <br>
Carlos José Guerrero Silva - carlosj-guerrero@javeriana.edu.co <br>
Manuel Alejandro Huérfano Castro - huerfanom@javeriana.edu.co <br>
Cesar Augusto González Cortes - ca.gonzalezc@javeriana.edu.co

In [40]:
import pandas as pd
import requests
import json

Se carga el archivo CSV, previamente se habia descargardo el archivo que originalmente esta en Excel y se guardo como CSV, posterior a esto se deja en una cuenta de Github para facilitar la carga

In [41]:
# Cargar el archivo CSV desde la URL con el delimitador para ';' subida en github
url_csv = 'https://raw.githubusercontent.com/drmanuelhuerfano/SalidasColombianos/main/Salidas_de_colombianos_desde_el_territorio_nacional.csv'
salidas_df = pd.read_csv(url_csv, delimiter=';')

# Mostrar las primeras filas del DataFrame para verificar la carga correcta
print(salidas_df.head())

   Year    Mes Pais-destino  Codigo Iso 3166  Femenino  Masculino  Indefinido  \
0  2012  Enero   Afganistan                4         0          1         NaN   
1  2012  Enero   Afganistan                4         1         20         NaN   
2  2012  Enero     Alemania              276         1          0         NaN   
3  2012  Enero     Alemania              276         5          2         NaN   
4  2012  Enero     Alemania              276         3          2         NaN   

   Total      Latitud - Longitud  
0      1  (10.886611,-74.776708)  
1     21   (4.697144,-74.140516)  
2      1   (0.814836,-77.662532)  
3      7  (10.445761,-75.516429)  
4      5  (10.886611,-74.776708)  


In [42]:
# Mostrar todas las columnas del DataFrame para verificar los nombres
print(salidas_df.columns)

Index(['Year', 'Mes', 'Pais-destino', 'Codigo Iso 3166', 'Femenino',
       'Masculino', 'Indefinido', 'Total', 'Latitud - Longitud'],
      dtype='object')


In [43]:
# Verificar el tipo de dato de una columna específica
tipo_columna = salidas_df['Codigo Iso 3166'].dtype
print(f"El tipo de dato de la columna 'Codigo Iso 3166' es: {tipo_columna}")


El tipo de dato de la columna 'Codigo Iso 3166' es: int64


Se convierte a cadena de texto para que no exista conflicto despues con el diccionario que sale del JSON

In [44]:
# Convertir la columna 'Codigo Iso 3166' a tipo cadena de texto
salidas_df['Codigo Iso 3166'] = salidas_df['Codigo Iso 3166'].astype(str)

Se carga el JSON desde la API Restcountries.com

In [45]:
# Hacer la solicitud a la API de REST Countries para obtener todos los países
url = 'https://restcountries.com/v3.1/all'
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    countries_data = response.json()
    print("Datos de la API cargados exitosamente")
else:
    print("Error al cargar los datos de la API:", response.status_code)

# Crear un DataFrame a partir de los datos de la API
countries_df = pd.json_normalize(countries_data)

# Mostrar las columnas disponibles en el DataFrame para entender su estructura
print(countries_df.columns)


Datos de la API cargados exitosamente
Index(['tld', 'cca2', 'ccn3', 'cca3', 'independent', 'status', 'unMember',
       'capital', 'altSpellings', 'region',
       ...
       'name.nativeName.heb.common', 'languages.heb',
       'name.nativeName.mri.official', 'name.nativeName.mri.common',
       'name.nativeName.nzs.official', 'name.nativeName.nzs.common',
       'languages.mri', 'languages.nzs', 'currencies.NIO.name',
       'currencies.NIO.symbol'],
      dtype='object', length=896)


Ahora se crea un Diccionario para Mapear Países a Subregiones

In [46]:
# Crear un diccionario para mapear cada país a su subregión
country_to_subregion = countries_df.set_index('name.common')['subregion'].to_dict()

# Mostrar una muestra del diccionario para verificar
print(list(country_to_subregion.items())[:10])


[('South Georgia', nan), ('Grenada', 'Caribbean'), ('Switzerland', 'Western Europe'), ('Sierra Leone', 'Western Africa'), ('Hungary', 'Central Europe'), ('Taiwan', 'Eastern Asia'), ('Wallis and Futuna', 'Polynesia'), ('Barbados', 'Caribbean'), ('Pitcairn Islands', 'Polynesia'), ('Ivory Coast', 'Western Africa')]


Como vamos a trabajar con los codigos ISO 3166 vamos a crear una lista en formato JSON con las columnas de interés obtenidas de la API de REST Countries.

In [47]:
# Crear una lista para almacenar el JSON con las columnas de interés
json_list = []

# Recorrer los datos de la API de REST Countries
for entry in countries_data:
    # Añadir los elementos al JSON solo si la entrada contiene los campos necesarios
    if 'name' in entry and 'common' in entry['name'] and 'subregion' in entry and 'ccn3' in entry:
        json_list.append({
            'Pais destino': entry['name']['common'],   # Nombre común del país
            'subregion': entry['subregion'],           # Subregión
            'Codigo Iso 3166': entry['ccn3']           # Código ISO 3166
        })

# Mostrar el JSON creado
print("El nuevo JSON es:\n")
print(json_list)


El nuevo JSON es:

[{'Pais destino': 'Grenada', 'subregion': 'Caribbean', 'Codigo Iso 3166': '308'}, {'Pais destino': 'Switzerland', 'subregion': 'Western Europe', 'Codigo Iso 3166': '756'}, {'Pais destino': 'Sierra Leone', 'subregion': 'Western Africa', 'Codigo Iso 3166': '694'}, {'Pais destino': 'Hungary', 'subregion': 'Central Europe', 'Codigo Iso 3166': '348'}, {'Pais destino': 'Taiwan', 'subregion': 'Eastern Asia', 'Codigo Iso 3166': '158'}, {'Pais destino': 'Wallis and Futuna', 'subregion': 'Polynesia', 'Codigo Iso 3166': '876'}, {'Pais destino': 'Barbados', 'subregion': 'Caribbean', 'Codigo Iso 3166': '052'}, {'Pais destino': 'Pitcairn Islands', 'subregion': 'Polynesia', 'Codigo Iso 3166': '612'}, {'Pais destino': 'Ivory Coast', 'subregion': 'Western Africa', 'Codigo Iso 3166': '384'}, {'Pais destino': 'Tunisia', 'subregion': 'Northern Africa', 'Codigo Iso 3166': '788'}, {'Pais destino': 'Italy', 'subregion': 'Southern Europe', 'Codigo Iso 3166': '380'}, {'Pais destino': 'Benin'

In [48]:
# Crear un diccionario para mapear el código ISO 3166-1 a la subregión
iso_to_subregion = {entry['Codigo Iso 3166']: entry['subregion'] for entry in json_list}

# Mostrar una muestra del diccionario para verificar
print(list(iso_to_subregion.items())[:10])


[('308', 'Caribbean'), ('756', 'Western Europe'), ('694', 'Western Africa'), ('348', 'Central Europe'), ('158', 'Eastern Asia'), ('876', 'Polynesia'), ('052', 'Caribbean'), ('612', 'Polynesia'), ('384', 'Western Africa'), ('788', 'Northern Africa')]


Utilizando la columna 'Codigo Iso 3166' del DataFrame salidas_df se mapea la subregión correspondiente a cada país utilizando el diccionario iso_to_subregion, que se construyó previamente a partir del JSON descargado de la API de REST Countries.

In [49]:
# Asegurarse de que las claves del diccionario 'iso_to_subregion' también son cadenas de texto
iso_to_subregion = {str(k): v for k, v in iso_to_subregion.items()}

# Aplicar el mapeo  con el tipo de dato corregido
salidas_df['subregion'] = salidas_df['Codigo Iso 3166'].map(iso_to_subregion)

# Verificar que las subregiones hayan sido asignadas correctamente
print(salidas_df.head())

   Year    Mes Pais-destino Codigo Iso 3166  Femenino  Masculino  Indefinido  \
0  2012  Enero   Afganistan               4         0          1         NaN   
1  2012  Enero   Afganistan               4         1         20         NaN   
2  2012  Enero     Alemania             276         1          0         NaN   
3  2012  Enero     Alemania             276         5          2         NaN   
4  2012  Enero     Alemania             276         3          2         NaN   

   Total      Latitud - Longitud       subregion  
0      1  (10.886611,-74.776708)             NaN  
1     21   (4.697144,-74.140516)             NaN  
2      1   (0.814836,-77.662532)  Western Europe  
3      7  (10.445761,-75.516429)  Western Europe  
4      5  (10.886611,-74.776708)  Western Europe  


Subregiones NaN

In [50]:
# Identificar códigos ISO que no pudieron ser mapeados
codigos_no_mapeados = salidas_df[salidas_df['subregion'].isna()]['Codigo Iso 3166'].unique()

print("Códigos ISO que no pudieron ser mapeados:")
print(codigos_no_mapeados)


Códigos ISO que no pudieron ser mapeados:
['4' '20' '24' '28' '12' '32' '36' '40' '31' '44' '48' '50' '52' '56' '84'
 '60' '68' '72' '76' '96' '99999' '736' '92' '8' '51' '70' '90' '16' '10'
 '64' '334' '86']


Se completa el  Diccionario de Mapeo Manualmente
Se colocan los códigos ISO faltantes al diccionario iso_to_subregion.

In [51]:
# Completar el diccionario con los códigos ISO que no fueron mapeados
iso_to_subregion.update({
    '4': 'Southern Asia',           # Afghanistan
    '20': 'Northern Africa',        # Andorra
    '24': 'Sub-Saharan Africa',     # Angola
    '28': 'Caribbean',              # Antigua and Barbuda
    '12': 'Northern Africa',        # Algeria
    '32': 'South America',          # Argentina
    '36': 'Australia and New Zealand', # Australia
    '40': 'Western Europe',         # Austria
    '31': 'Western Asia',           # Azerbaijan
    '44': 'Caribbean',              # Bahamas
    '48': 'Western Asia',           # Bahrain
    '50': 'Southern Asia',          # Bangladesh
    '52': 'Caribbean',              # Barbados
    '56': 'Western Europe',         # Belgium
    '84': 'Central America',        # Belize
    '60': 'Caribbean',              # Bermuda
    '68': 'South America',          # Bolivia
    '72': 'Sub-Saharan Africa',     # Botswana
    '76': 'South America',          # Brazil
    '96': 'South-Eastern Asia',     # Brunei
    '736': 'Northern Africa',       # Sudan
    '92': 'Caribbean',              # British Virgin Islands
    '8': 'Southern Europe',         # Albania
    '51': 'Western Asia',           # Armenia
    '70': 'Southern Europe',        # Bosnia and Herzegovina
    '90': 'Melanesia',              # Solomon Islands
    '16': 'Polynesia',              # American Samoa
    '10': 'Antarctica',             # Antarctica
    '64': 'Southern Asia',          # Bhutan
    '334': 'Antarctica',            # Bouvet Island
    '86': 'Antarctica',             # Heard Island and McDonald Islands
   })

Se aplica de Nuevo el Mapeo y se verifica ausencias de NaN

In [52]:
# Aplicar el mapeo para obtener la subregión usando el código ISO 3166
salidas_df['subregion'] = salidas_df['Codigo Iso 3166'].map(iso_to_subregion)

# Verificar que las subregiones hayan sido asignadas correctamente
print(salidas_df.head())

   Year    Mes Pais-destino Codigo Iso 3166  Femenino  Masculino  Indefinido  \
0  2012  Enero   Afganistan               4         0          1         NaN   
1  2012  Enero   Afganistan               4         1         20         NaN   
2  2012  Enero     Alemania             276         1          0         NaN   
3  2012  Enero     Alemania             276         5          2         NaN   
4  2012  Enero     Alemania             276         3          2         NaN   

   Total      Latitud - Longitud       subregion  
0      1  (10.886611,-74.776708)   Southern Asia  
1     21   (4.697144,-74.140516)   Southern Asia  
2      1   (0.814836,-77.662532)  Western Europe  
3      7  (10.445761,-75.516429)  Western Europe  
4      5  (10.886611,-74.776708)  Western Europe  


In [53]:
# Identificar códigos ISO que no pudieron ser mapeados
codigos_no_mapeados = salidas_df[salidas_df['subregion'].isna()]['Codigo Iso 3166'].unique()

print("Códigos ISO que no pudieron ser mapeados:")
print(codigos_no_mapeados)

Códigos ISO que no pudieron ser mapeados:
['99999']


Salida de un DataFrame con las siguientes columnas:
**año,subregion,total**
En donde la subregión es la subregión del mundo hacia la cuál salieron los colombianos en ese año, y total es el total de viajes registrados por Migración Colombia durante el año

In [54]:
# Agrupar los datos por año y subregión, sumando el total de salidas
aggregated_df = salidas_df.groupby(['Year', 'subregion']).agg(total=('Total', 'sum')).reset_index()

# Renombrar las columnas para cumplir con el formato solicitado
aggregated_df.columns = ['año', 'subregion', 'total']

# Mostrar el DataFrame resultante
print(aggregated_df.head())

# Guardar el DataFrame en un archivo CSV
output_path = 'resultado_agregado_salidas.csv'
aggregated_df.to_csv(output_path, sep=';', index=False)

print(f"Archivo guardado exitosamente en: {output_path}")

    año                  subregion   total
0  2012  Australia and New Zealand    6190
1  2012                  Caribbean  121803
2  2012            Central America  409359
3  2012               Central Asia      35
4  2012             Central Europe     791
Archivo guardado exitosamente en: resultado_agregado_salidas.csv
